In [ ]:
import sys
import os
import time
import logging
import datetime
from urllib.parse import urlencode
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from google.cloud import bigquery
from google.cloud import storage
import json
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import importlib
import aiohttp
import asyncio
import flask

sys.path.append('../GitHub/core-functions/src/dreams_core')
import googlecloud as dgc2
importlib.reload(dgc2)


# sys.path.append('../GitHub/etl-pipelines/cloud_functions/core_chains')
# import main as wip
# importlib.reload(wip)

load_dotenv()

logger = dc.setup_logger()
logger.setLevel(logging.DEBUG)

# Custom format function for displaying numbers
pd.set_option('display.float_format', lambda x: f'{x:.15g}')
# pd.reset_option('display.float_format')


#### Syntax to trigger google cloud functions

## Queries with notes

### token metadata

In [ ]:
-- successfully retrieves metadata for MILKBAG

SELECT
*
FROM
  -- `bigquery-public-data.crypto_solana_mainnet_us.Token Transfers` t
  `bigquery-public-data.crypto_solana_mainnet_us.Tokens` t
WHERE
  t.block_timestamp between '2024-03-01' and '2024-04-30'
 and mint='2ubuHGFS4VJVxSEpvV3kDwz6JiuXdaAoGMwrwYC87tp8'

In [ ]:
[{
  "block_slot": "255513963",
  "block_hash": "3JxMLj4B5WoGPYhM6oEQR2jZ3qytWuUQtyKZALZtNwZD",
  "block_timestamp": "2024-03-21 08:51:38.000000 UTC",
  "tx_signature": "5R6bGVgcByPEz6qeX5MhJCBreiMAQBz4NSSRrTBDJzqow4RQP3DHsTZMZAvTfFuFYVcBx2XNp2TWdjVfedN7dj7k",
  "retrieval_timestamp": "2024-03-21 22:12:36.000000 UTC",
  "is_nft": "false",
  "mint": "2ubuHGFS4VJVxSEpvV3kDwz6JiuXdaAoGMwrwYC87tp8",
  "update_authority": "FkNS1zBQrsb9a4PAvfBArGNcjzwZjEqEnLm7YMr5xqY8",
  "name": "MILKBAG",
  "symbol": "MILKBAG",
  "uri": "https://bafkreibmwrmuk4x4gawfluqfvmjjfvsq5gwblyn3cnwknmnmcw5l32etsy.ipfs.nftstorage.link/",
  "seller_fee_basis_points": "0",
  "creators": [{
    "address": null,
    "verified": null,
    "share": null
  }],
  "primary_sale_happened": "false",
  "is_mutable": "false"
}]

### transfer counts

In [ ]:
SELECT
count(*)
FROM
  `bigquery-public-data.crypto_solana_mainnet_us.Token Transfers` t
  -- `bigquery-public-data.crypto_solana_mainnet_us.Tokens` t
WHERE
  t.block_timestamp between '2024-03-01' and '2024-03-31'
 and mint='2ubuHGFS4VJVxSEpvV3kDwz6JiuXdaAoGMwrwYC87tp8'

# transaction types data assessment

## High level assessment

A pipeline to retrieve accurate transfers data should be buildable using the bigquery components, but will require different treatments for transfer, burn, and mint transactions. A key concept to understand is that solana differentiates accounts (token-specific adddresses) from owners (general addresses that own an account for their token transactions). One owner can have many accounts, and all accounts have an owner. 

Transaction success/failure is only available in the bigquery Transactions table which is 346 billion records. It is unclear how failed transactions could be filtered from the dataset without querying this table, or querying dune for every single transaction. 

#### Transfers

* Sender: account address is available
* Receiver: account address is available
* Token Address: available
* Amount: available
* Timestamp: available

A crosswalk from account to owner would need to be made. 


#### Burns

* Sender: owner address is available
* Receiver: not applicable
* Token Address: available
* Amount: available
* Timestamp: available

#### Mints

* Sender: not applicable
* Receiver: no information available
* Token Address: available
* Amount: available
* Timestamp: available

the receiver info would have to be extracted from dune. it could potentially be in the bigquery transactions table but that is massive and would be expensive to incorporate. 



## spl-transfer

34649/65166 records

Key Fields

* sender address: source
* receiver address: destination
* token address: mint
* amount: value

Note that these refer to the address rather than the owner. Burn records show the owner. A crosswalk should be able to be created via either the bigquery Accounts table or via Dune. 

In [ ]:
pd.set_option('display.max_colwidth', None)  # or specify a number for a custom limit

query_sql = """
    select *
    from `sandbox.solana_transfers_2023_05`
    where transfer_type = 'spl-transfer'
"""

transfer_df = dgc().run_sql(query_sql)
transfer_df.head()

## burn

30426/65166 records

### Key Fields
* sender account owner: authority
* token address: mint
* amount: value

### ETL Feasibility
we should have enough data in this table to generate transfers calculations

In [ ]:
query_sql = """
    select *
    from `sandbox.solana_transfers_2023_05`
    where transfer_type = 'burn'
"""

burn_df = dgc().run_sql(query_sql)
burn_df.head()
# burn_df['tx_signature'][0]

## mintTo 

91/65166 records

the data from bigquery does not include anything about who received the tokens, and will need to be matched to another source in order to obtain the receiving wallet address. 

In [ ]:
query_sql = """
    select *
    from `sandbox.solana_transfers_2023_05`
    where transfer_type = 'mintTo'
"""

mint_df = dgc().run_sql(query_sql)
mint_df.head()

### ETL feasibility

the biggest risk is that there is a problem with either bigquery or dune records, or that the joins between the two are somehow imperfect. as of now there are 7347 total mint transactions that would need to be retrieved for all solana core.coins which means that the total volume of extractions would be very low compared to transfers pipelines. 

In [ ]:
# dune count query

dune_sql = """
    with dreams_tokens as (
        select token_address
        from dune.dreamslabs.etl_net_transfers_freshness t
        where chain = 'solana'
        group by 1
    )

    select count(*)
    from tokens_solana.transfers t
    join dreams_tokens dt on dt.token_address = t.token_mint_address
    where action = 'mint'
"""

### matching mint transfers to dune

the bigquery table does not provide information about the token receiver. we have the transaction signature, token address (mint), and amount (value) but there is no information about who receives the tokens as the destination and authority fields are all empty. 

the transaction signature can be matched to dune where the receiving account owner data is available. 

match fields:
* bq.mint = dune.token_mint_address
* bq.tx_signature = dune.tx_id
* bq.value = dune.amount

wallet_address_receiver
* dune.to_owner

#### example record match

In [ ]:
# bigquery data

tx = '3RhHy1p2p5uAw7QDFjuNwWUQqtuCwfwFJ9BQLNE3UBHmZgpTdWCECGzzFJ8qkGnuoZvQpNZNiSoeXz97DRm64azq'
token = 'hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux'
mint_df[
    (mint_df['tx_signature']==tx)
    & (mint_df['mint']==token)
]

In [ ]:
# dune data

dune_query = f"""
    select *
    from tokens_solana.transfers t
    where block_date = cast('2023-05-01' as date)
    and tx_id = {tx}
    and token_mint_address = {token}
    and action = 'mint'
"""

pd.read_csv('solana_transfer_analysis/mintTo_dune_sample.csv')

In [ ]:
##

## export data from public bigquery to GCS

In [ ]:
EXPORT DATA
OPTIONS(
  uri='gs://dreams-labs-storage/data_lake/solana_transfers/dev_2023_05/2023_05_*.csv',
  format='CSV',
  overwrite=true
) AS

-- SELECT tx_signature
-- ,block_timestamp
-- ,source
-- ,destination
-- ,mint
-- ,value
select *
FROM
  `bigquery-public-data.crypto_solana_mainnet_us.Token Transfers` t
WHERE
  t.block_timestamp between '2023-05-01' and '2023-05-31'
and t.mint in (
'Cbet5zMSzpCUKmVfy4nSkBjGPSy7WoUEwqLcQ6HhrHVA'
,'HovGjrBGTfna4dvg6exkMxXuexB3tUfEZKcut8AWowXj','7SdFACfxmg2eetZEhEYZhsNMVAu84USVtfJ64jFDCg9Y','25hAyBQfoDhfWx9ay6rarbgvWGwDdNqcHsXS3jQ3mTDJ','D1nj2nyuVLHtL1Fd96hXzhUgaet9c9LTvXRs7E2Rpump','4XQvdipJBdrb5hUgUrbZPPFmp6BCav41n55dc7KDYW3m','A3eME5CetyZPBoWbRUwY3tSe25S6tb18ba9ZPbWk9eFJ','2ez1pFrqmsXa4688qMQezgxGq7cDWhKPebJTHPSdUXDY','5mbK36SZ7J19An8jFochhQS4of8g6BwUjbeCSxBSoWdp','orcaEKTdK7LKz57vaAYr9QeNsVEPfiu6QeMU1kektZE','ZhiSHHegARupUWNj8nNoV9Q8CgTpYLz79U3m3UF3r9b','8mq2np5SgMpJxZeNXjeoVYvjNWAnGKhYJU2Xj4GxFz5Q','HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3RKwX8eACQBCt3','45EgCwcPXYagBC7KqBin4nCFgEZWN7f3Y6nACwxqMCWX','26KMQVgDUoB6rEfnJ51yAABWWJND8uMtpnQgsHQ64Udr','4G3kNxwaA2UQHDpaQtJWQm1SReXcUD7LkT14v2oEs7rV','EjErrBoCw7eWYkMfimhPckaPSuBukyhUYwv2dLJYDWB7','EHCwJQi8dSpZfKm4LJypzozEj5vAN7pESRXJGpESKMfJ','6yjNqPzTSanBWSa6dxVEgTjePXBrZ2FoHLDQwYwEsyM6','FYa25XnBsXQXAdTnsyKBKd5gZ1VZhChBRF57CqfRxJZX','6tWuipcDv4CiHtXid7JctDbtLLA1VdkTwSDupK6UxzJL','8doS8nzmgVZEaACxALkbK5fZtw4UuoRp4Yt8NEaXfDMb','9BBd5VJUPK41ntmdEvBMdGg2aXSnDpgVYxcDwP5c78Ym','69kdRLyP5DTRkpHraaSZAQbWmAwzF9guKjZfzMXzcbAs','C1kzNkFfgdtP8VF1pFYA4S32RLPqk5KPaurCaQJwxfWb','E2BGnzHdJNUBtAVR7EyQMuEMHqgv65JL8J9ZyqyXUVvA','EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm','5LafQUrVco6o7KMz42eqVEJ9LW31StPyGjeeu5sKoMtA','4pb6mymm9hYQN6og9uF24eyZ2qwXCWCwGvcR1DkCgeEr','9XRpjZjhJPeWtUymiEWn3FW7uAnMeQca14ucTWWWyP2g','7D7BRcBYepfi77vxySapmeqRNN1wsBBxnFPJGbH5pump','8m9fjYycXAFva1kScttQgsESVZT7yELhjZASqfHBuMa5','ukHH6c7mMyiWCf1b9pnWe25TSpkDDt3H5pQZgZ74J82','3de2yRhtD4VbJBb8EQAQffYMPLU4EnSHT1eveBwiL3tn','AVLhahDcDQ4m4vHM4ug63oh7xc8Jtk49Dm5hoe9Sazqr','SNApmcWQqj3Ny2YFkQmkELQnNgaXRu6KmnYSPiFZcLn','GEdBv2DnES89DvasmZ35TaxP9kBibznYKbacXpoGTEBU','pawSXHWsonrTey4SX7tz1fM9ksuLpE13Y54K57ym4Rg','nosXBVoaCTtYdLvKY6Csb4AC8JCdQKKAaWYtx2ZMoo7','7vuhsRQ2gE4WPv37qegBKu8PcWHxDb5rQ6fQKkDfUghF','BNT4uhSStq1beFADv3cq4wQAVfWB392PjAaxTBpNeWxu','B5LMXiuvbB5jN3auECUtdfyeFWm27krgFinrBrqJGFRM','Adq3wnAvtaXBNfy63xGV1YNkDiPKadDT469xF9uZPrqE','7iT1GRYYhEop2nV1dyCwK2MGyLmPHq47WhPGSwiqcUg5','hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux','HuPspkki5Qdnf5WAU7jtEThkeMhni6XQ23tunZRkZWUi','754Ry9yULcPgSHcmfLSQDiihZgs7917dJUZ9513FLkg7','DeoP2swMNa9d4SGcQkR82j4RYYeNhDjcTCwyzEhKwfAf','SHDWyBxihqiCj6YekG2GUr7wqKLeLAMK1gHZck9pL6y','9V4x6ikFm9XKsnh3TiYJWPwQfFkJZDjifu7VSUqg3es1','BWXWbFu8bYtJRrDb4bRpaSPz8PQZvTG8ZK5bwkPWhgcJ','SNSNkV9zfG5ZKWQs6x4hxvBRV6s8SqMfSGCtECDvdMd','BGyjasmSzYM9hHiZ1LBU4EJ7KCtRjMSpbN4zTru3W5vf','CdZaJzc2BdmHhbr3LTP4DCPKyBu4zJrfB2mQKCgURUgp','NeonTjSjsuo3rexg9o6vHuMXw62f9V7zvmu8M8Zut44','8wXtPeU6557ETkp9WHFY1n1EcU6NxDvbAggHGsMYiHsB','jtojtomepa8beP8AuQc6eXt5FriJwfFMwQx2v2f9mCL','G33s1LiUADEBLzN5jL6ocSXqrT2wsUq9W6nZ8o4k1b4L','7GCihgDB8fe6KNjn2MYtkzZcRjQy3t9GHdC8uHYmW2hr','octo82drBEdm8CSDaEKBymVn86TBtgmPnDdmE64PTqJ','3ag1Mj9AKz9FAkCQ6gAEhpLSX8B2pUbPdkb9iBsDLZNB','MNDEFzGvMt87ueuHvVU9VcTqsAP5b3fTGPsHuuPA5ey','FoVeWwe6H6hWEa1cQfZeNSGsGSDyKz57CZT49BusdshW','DcUoGUeNTLhhzyrcz49LE7z3MEFwca2N9uSw1xbVi1gm','PUPS8ZgJ5po4UmNDfqtDMCPP6M1KP3EEzG9Zufcwzrg','5z3EqYQo9HiCEs3R84RCDMu2n7anpDMxRhdK8PSWmrRC','GtDZKAqvMZMnti46ZewMiXCa4oXF4bZxwQPoKzXPFxZn','52DfsNknorxogkjqecCTT3Vk2pUwZ3eMnsYKVm4z3yWy','FU1q8vJpZNUrmqsciSjp8bAKKidGsLmouB8CBdf8TKQv','BSHanq7NmdY6j8u5YE9A3SUygj1bhavFqb73vadspkL3','Avp2VDgnQqxsnrjtq3ynNhKCfWGEGj1PmGLY5ZmgonjH','7njsg9BA1xvXX9DNpe5fERHK4zb7MbCHKZ6zsx5k3adr','DdqUGjhtZ8uNU7YHRsNFwXL5qM8Dbyiuzm22DRheN3aK','2ubuHGFS4VJVxSEpvV3kDwz6JiuXdaAoGMwrwYC87tp8','ULwSJmmpxmnRfpu6BjnK6rprKXqD5jXUmPpS1FxHXFy','4tJZhSdGePuMEfZQ3h5LaHjTPsw1iWTRFTojnZcwsAU6','GDfnEsia2WLAW5t8yx2X5j2mkfA74i5kwGdDuZHt7XmG','EJPtJEDogxzDbvM8qvAsqYbLmPj5n1vQeqoAzj9Yfv3q','Hf5gAgohzfUyjytaF5aUSMDwsPAbdThQJNnqw97reGMw','EsirN3orp85uyvZyDrZnbe9cyo7N1114ynLFdwMPCQce','3bRTivrVsitbmCTGtqwp7hxXPsybkjn4XLNtPsHqa3zR','BiDB55p4G3n1fGhwKFpxsokBMqgctL4qnZpDH1bVQxMD','947tEoG318GUmyjVYhraNRvWpMX7fpBTDQFBoJvSkSG3','AfcvNFud8cQPKpCZtW8GBsJqi2LJNztFPu8d4vciveL3','5ritAPtFPqQtEFHcHVqNjR5oFNUJqcmgKtZyPd2AyLLy','9niFQK8MsParjSxhYQ3Ys2a6zHmsEuKSB1M7wwdvZ7bj')

## extract GCS data to project bigquery

In [ ]:
import os
from google.cloud import storage
import pandas as pd
from io import StringIO

def combine_csv_files(bucket_name, prefix):
    # Initialize the GCS client
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # List all blobs with the given prefix
    blobs = bucket.list_blobs(prefix=prefix)

    # Column names
    colnames = [
        'block_slot', 'block_hash', 'block_timestamp', 'tx_signature',
        'source', 'destination', 'authority', 'value', 'decimals',
        'mint', 'mint_authority', 'fee', 'fee_decimals', 'memo', 'transfer_type'
    ]

    # List to store all dataframes
    dfs = []

    # Iterate through all blobs
    for blob in blobs:
        if blob.name.endswith('.csv'):
            # Download the content of the blob
            content = blob.download_as_text()

            # Create a DataFrame from the content
            df = pd.read_csv(StringIO(content), names=colnames, header=None)

            # Only append if the DataFrame is not empty
            if not df.empty:
                dfs.append(df)

            print(f"Processed {blob.name}")

    # Combine all dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

# Usage
bucket_name = "dreams-labs-storage"
prefix = "data_lake/solana_transfers/dev_2023_05/"

result_df = combine_csv_files(bucket_name, prefix)

# # Optional: Save the combined DataFrame to a local CSV file
# result_df.to_csv("combined_solana_transfers.csv", index=False)

print(f"Combined DataFrame shape: {result_df.shape}")

In [ ]:
result_df.head()

In [ ]:
import os
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
from io import StringIO


def upload_to_bigquery(df, project_id, dataset_id, table_id):
    # Initialize BigQuery client
    client = bigquery.Client(project=project_id)

    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)

    # Define the job config
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition="WRITE_TRUNCATE",  # This will overwrite the table if it exists
    )

    # Load the dataframe into BigQuery
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

    # Wait for the job to complete
    job.result()

    print(f"Loaded {job.output_rows} rows into {project_id}:{dataset_id}.{table_id}")


# Upload to BigQuery
project_id = "western-verve-411004"  # Replace with your Google Cloud project ID
dataset_id = "sandbox"  # Replace with your BigQuery dataset ID
table_id = "solana_transfers_2023_05"   # Replace with your desired table name

upload_to_bigquery(result_df, project_id, dataset_id, table_id)

# Optionally, you can still save the combined DataFrame to a local CSV file
# result_df.to_csv("combined_solana_transfers.csv", index=False)